In [91]:
import pandas as pd
import numpy as np
import sys
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


In [74]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('data.csv')

df.drop(columns = ['season', 'seas_id', 'player', 'player_id', 'birth_year', 'lg'], axis = 1, inplace=True)

# cat_vars = ['pos']
# df = pd.get_dummies(data = df, prefix = cat_vars, columns = cat_vars)

df = df.dropna(inplace=False)
df


,pos,age,experience,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,SG,24.0,2,MIL,56,0.0,614.0,83,196,0.423,69.0,169.0,0.408,14,27,0.519,0.599,17,19,0.895,9.0,55.0,64.0,30,9.0,4.0,12.0,49,252
1,SG,23.0,2,DAL,42,0.0,311.0,54,121,0.446,13.0,50.0,0.260,41,71,0.577,0.500,15,23,0.652,14.0,36.0,50.0,20,10.0,3.0,14.0,22,136
2,SF,20.0,2,ATL,20,0.0,171.0,18,62,0.290,10.0,39.0,0.256,8,23,0.348,0.371,2,2,1.000,2.0,16.0,18.0,5,1.0,2.0,8.0,6,48
3,PF,28.0,10,DEN,73,73.0,2297.0,398,716,0.556,40.0,138.0,0.290,358,578,0.619,0.584,177,269,0.658,174.0,297.0,471.0,259,56.0,45.0,105.0,142,1013
4,PG,27.0,6,HOU,78,1.0,1269.0,186,417,0.446,84.0,217.0,0.387,102,200,0.510,0.547,58,63,0.921,23.0,100.0,123.0,140,42.0,6.0,53.0,125,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,SG,32.0,10,TOT,43,4.0,964.0,244,543,0.449,10.0,15.0,0.667,234,528,0.443,0.459,91,105,0.867,17.0,61.0,78.0,83,24.0,6.0,82.0,79,589
23782,SG,32.0,10,BOS,26,4.0,442.0,123,249,0.494,3.0,4.0,0.750,120,245,0.490,0.500,50,55,0.909,10.0,28.0,38.0,29,9.0,2.0,37.0,49,299
23803,C,24.0,2,BOS,82,27.0,1918.0,379,727,0.521,0.0,1.0,0.000,379,726,0.522,0.521,184,269,0.684,209.0,321.0,530.0,92,53.0,15.0,151.0,244,942
23843,SF,32.0,10,PHI,81,5.0,1543.0,363,703,0.516,4.0,10.0,0.400,359,693,0.518,0.519,207,249,0.831,114.0,176.0,290.0,149,67.0,9.0,132.0,114,937


In [82]:
X = df.drop(columns = ['pos', 'tm'])
y = df['pos']

y = y.dropna()
X = X.loc[y.index]


position_mapping = {
    'C': 0,
    'C-PF': 1,
    'PF': 2,
    'PF-C': 3,
    'PF-SF': 4,
    'PG': 5,
    'PG-SF': 6,
    'PG-SG': 7,
    'SF': 8,
    'SF-PF': 9,
    'SF-SG': 10,
    'SG': 11,
    'SG-PF': 12,
    'SG-PG': 13,
    'SG-PG-SF': 14,
    'SG-SF': 15,
}

# Encode using the mapping
y_encoded = y.map(position_mapping)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=1)

In [84]:
xg_model = xgb.XGBClassifier( eval_metric = 'mlogloss')
xg_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [85]:
y_pred_xg = xg_model.predict(X_test)

In [99]:
# Create a boolean mask for non-NaN values
mask = ~y_test.isna()

# Filter y_test and y_pred_xg based on the mask
y_test_clean = y_test[mask]
y_pred_clean = y_pred_xg[mask]

# Check for NaNs again
print(y_test_clean.isna().sum())


0


In [100]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test_clean, y_pred_clean)
print("Accuracy:", accuracy)


Accuracy: 0.636004217185029
